In [1]:
import torch
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

TODOs:
- turn into class functions: (the ones needed to estimate order)
- add as utils (computing of weigthed outdegrees and transition probabobilities)
- unit tests


In [51]:
# dag_data = pp.DAGData(pp.IndexMap(list("abcde")))
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

# walk_1  =('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a')
# walk_2  =('a','b','c','d','e','c')
# dag_data.append_walk(walk_1)
# dag_data.append_walk(walk_2)


dag_data.append_walk(list("0230230230230"), weight=2)
dag_data.append_walk(list("1241241241241"), weight=2)
# dag_data.append_walk(list("0430241"), weight=1)

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=5)

m.estimate_order(dag_data, max_order=5)

2

In [5]:
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])

In [41]:
walk_data = pp.DAGData(pp.IndexMap(list("abcd")))
walk_data.append_walk(("a","b","c","d"))
m = pp.MultiOrderModel.from_DAGs(walk_data, max_order=2)
# m.get_mon_dof(assumption="paths", max_order=0)

In [45]:
# DOF computation fails like this...
toy_paths = pp.DAGData(pp.IndexMap(list("abcde")))
toy_paths.append_walk(("a","c","d"))
toy_paths.append_walk(("b","c","e"))
toy_paths.append_walk(("a","c","e"))
toy_paths.append_walk(("b","c","d"))
max_order = 2
m = pp.MultiOrderModel.from_DAGs(toy_paths, max_order=max_order, mode = "propagation")
for order in range(max_order+1): 
    print(order)
    print(m.get_mon_dof(assumption="paths", max_order=order))

0
4
1
5
2


IndexError: index 4 is out of bounds for dimension 0 with size 4

In [46]:
dag_data = pp.DAGData(pp.IndexMap(list("abcde")))

dag_data.append_walk(tuple("acd"))
dag_data.append_walk(tuple("bce"))

m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=4)


def test_dof():
    line_data = pp.DAGData(pp.IndexMap(list("abcd")))
    line_data.append_walk(("a","b","c","d"))
    max_order = 1 #4
    m = pp.MultiOrderModel.from_DAGs(line_data, max_order=1)
    for o in range(max_order+1):
        assert m.get_mon_dof(assumption="paths", max_order=o) == 3
    

    toy_paths_ho = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths_ho.append_walk(("a","c","d"))
    toy_paths_ho.append_walk(("b","c","e"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths_ho, max_order=max_order, mode = "propagation")
    assert m.get_mon_dof(assumption="paths", max_order=0) == 4
    assert m.get_mon_dof(assumption="paths", max_order=1) == 5
    assert m.get_mon_dof(assumption="paths", max_order=2) == 7


def test_likelihood_ratio_test():
    pass

def test_estimate_order():
    pass

In [49]:
import numpy as np
def test_log_likelihood():
    toy_paths_ho = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths_ho.append_walk(("a","c","d"))
    toy_paths_ho.append_walk(("b","c","e"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths_ho, max_order=max_order, mode = "propagation")
    dag_graph = next(iter(DataLoader(toy_paths_ho.dags, batch_size=len(toy_paths_ho.dags)))).to(pp.config["torch"]["device"])
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=0),
        np.log(1/6) * 4 + np.log(2/6) * 2 
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=1),
        np.log(1/6) * 2 + 0 + 2 * np.log(1/2) 
        )
    # assert np.isclose(
    #     m.get_mon_log_likelihood(dag_graph, max_order=2)
    #     ,np.log(1/6) * 2 + 0 + 0 
    #     )
    ################

    toy_paths = pp.DAGData(pp.IndexMap(list("abcde")))
    toy_paths.append_walk(("a","c","d"))
    toy_paths.append_walk(("b","c","e"))
    toy_paths.append_walk(("a","c","e"))
    toy_paths.append_walk(("b","c","d"))
    max_order = 2
    m = pp.MultiOrderModel.from_DAGs(toy_paths, max_order=max_order, mode = "propagation")
    dag_graph = next(iter(DataLoader(toy_paths.dags, batch_size=len(toy_paths.dags)))).to(pp.config["torch"]["device"])
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=0), # fails already at computing log_lh here
        np.log(2/12) * 8 + np.log(4/12) * 4
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=1),
        np.log(2/12) * 4 + 0 + 4 * np.log(1/2)
        )
    assert np.isclose(
        m.get_mon_log_likelihood(dag_graph, max_order=2),
        np.log(1/6) * 4 + 0 + 4 * np.log(1/2)
        )
    
test_log_likelihood()


IndexError: index 9 is out of bounds for dimension 0 with size 8

In [50]:
toy_paths = pp.DAGData(pp.IndexMap(list("abcde")))
toy_paths.append_walk(("a","c","d"))
toy_paths.append_walk(("b","c","e"))
toy_paths.append_walk(("a","c","e"))
toy_paths.append_walk(("b","c","d"))
max_order = 2
m = pp.MultiOrderModel.from_DAGs(toy_paths, max_order=max_order, mode = "propagation")
dag_graph = next(iter(DataLoader(toy_paths.dags, batch_size=len(toy_paths.dags)))).to(pp.config["torch"]["device"])
m.get_mon_log_likelihood(dag_graph, max_order=0) # even zeroth order not working

IndexError: index 9 is out of bounds for dimension 0 with size 8

In [30]:
dag_graph.ptr

tensor([ 0,  3,  6,  9, 12])